In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV,KFold, cross_val_score
from sklearn.svm import LinearSVC
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.model_selection import KFold

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')
import lightgbm

[nltk_data] Downloading package stopwords to C:\Users\HU
[nltk_data]     ZEWEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\HU
[nltk_data]     ZEWEN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\HU
[nltk_data]     ZEWEN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\HU
[nltk_data]     ZEWEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\HU
[nltk_data]     ZEWEN\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def grid_search(model, parameters, features, labels):
  cv = GridSearchCV(model, parameters, cv=5)
  cv.fit(features, labels.values.ravel())
  print_results(cv)

  return cv

In [3]:
df_hotel = pd.read_csv("tripadvisor_hotel_reviews.csv")
positive = [4, 5]
neutral = [3]
negative = [1, 2]

def map_sentiment(rating):
    if rating in positive:
        return 2
    elif rating in neutral:
        return 1
    else:
        return 0

df_hotel['Sentiment']= df_hotel['Rating'].apply(map_sentiment)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000, tokenizer = word_tokenize)
X = tfidf.fit_transform(df_hotel['Review'])
y = df_hotel['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=24)

In [8]:
# Stacking 
estimators = [('rf', RandomForestClassifier()),
              ('ada', AdaBoostClassifier())]

sc = StackingClassifier(estimators=estimators)
sc.get_params()
parameters = {
    'rf__n_estimators': [500],
    'ada__n_estimators': [5000],
    'final_estimator': [LogisticRegression(C=10)],
    'passthrough': [False]
}
sc.fit(X_train, y_train)

predicted_sc = sc.predict(X_test)
accuracy_sc = accuracy_score(y_test, predicted_sc)
print('Accuracy:', accuracy_sc)
print('Classification Report:')
print(classification_report(y_test, predicted_sc))

Accuracy: 0.8489166504001562
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.76      0.75       810
           1       0.56      0.14      0.23       565
           2       0.88      0.97      0.92      3748

    accuracy                           0.85      5123
   macro avg       0.73      0.63      0.63      5123
weighted avg       0.82      0.85      0.82      5123

